In [1]:
import sys
sys.path.append("../../")
import time
import torch
from losses.basic_losses import CategoricalCrossentropyLoss, KLDivergenceLoss
from agents.random import RandomAgent
from agents.muzero import MuZeroAgent
from agent_configs.muzero_config import MuZeroConfig
from game_configs.variable_turn_tictactoe_config import VariableTurnTicTacToeConfig
from agents.tictactoe_expert import TicTacToeBestAgent
from modules.world_models.muzero_world_model import MuzeroWorldModel

# Ensure we use CPU for fairness/comparibility or GPU if available
device = "cpu" # or "cuda" if available
print(f"Using device: {device}")

/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
params = {
    "num_simulations": 25,
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "n_step": 10,
    "root_dirichlet_alpha": 0.25,
    "residual_layers": [(24, 3, 1)],
    "reward_dense_layer_widths": [],
    "reward_conv_layers": [(16, 1, 1)],
    "actor_dense_layer_widths": [],
    "actor_conv_layers": [(16, 1, 1)],
    "critic_dense_layer_widths": [],
    "critic_conv_layers": [(16, 1, 1)],
    "to_play_dense_layer_widths": [],
    "to_play_conv_layers": [(16, 1, 1)],
    "known_bounds": [-1, 1],
    "support_range": None,
    "minibatch_size": 8,
    "replay_buffer_size": 100000,
    "gumbel": False,
    "gumbel_m": 16,
    "policy_loss_function": CategoricalCrossentropyLoss(),
    "training_steps": 20000, # Reduced for benchmark speed
    "transfer_interval": 1,
    "num_workers": 4,
    "world_model_cls": MuzeroWorldModel,
    "search_batch_size": 0, # Iterative
    "use_virtual_mean": False,
    "virtual_loss": 3.0,
    "compile": True,
    "use_mixed_precision": True,
}

game_config = VariableTurnTicTacToeConfig()


In [ ]:
print("--- Running Multi Turn Tic Tac Toe ---")
params_batched = params.copy()
params_batched["search_batch_size"] = 5 
params_batched["use_virtual_mean"] = True

env_batch = VariableTurnTicTacToeConfig().make_env()
config_batch = MuZeroConfig(config_dict=params_batched, game_config=game_config)
config_batch.search_batch_size = 5 # Explicitly set

agent_batch = MuZeroAgent(
    env=env_batch,
    config=config_batch,
    name="variable_turn_tictactoe_muzero",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_batch.checkpoint_interval = 100
agent_batch.test_interval = 1000
agent_batch.test_trials = 100

start_time = time.time()
agent_batch.train()
end_time = time.time()
print(f"MuZero Batched Time: {end_time - start_time:.2f}s")

--- Running Multi Turn Tic Tac Toe ---
Using default save_intermediate_weights     : False
Using         training_steps                : 20000
Using default adam_epsilon                  : 1e-08
Using default momentum                      : 0.9
Using default learning_rate                 : 0.001
Using default clipnorm                      : 0
Using default optimizer                     : <class 'torch.optim.adam.Adam'>
Using default weight_decay                  : 0.0
Using default num_minibatches               : 1
Using default training_iterations           : 1
Using default lr_schedule_type              : none
Using default lr_schedule_steps             : []
Using default lr_schedule_steps             : []
Using default lr_schedule_values            : []
Using         use_mixed_precision           : True
Using         compile                       : True
Using default compile_mode                  : default
Using         minibatch_size                : 8
Using         replay_buffer_s

/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an AP

[Worker 0] Starting self-play...[Worker 3] Starting self-play...

[Worker 2] Starting self-play...[Worker 1] Starting self-play...

Started recording episode 0 to ./videos/variable_turn_tictactoe_muzero/0/episode_000000.mp4
Started recording episode 0 to ./videos/variable_turn_tictactoe_muzero/3/episode_000000.mp4Started recording episode 0 to ./videos/variable_turn_tictactoe_muzero/2/episode_000000.mp4

Started recording episode 0 to ./videos/variable_turn_tictactoe_muzero/1/episode_000000.mp4
Stopped recording episode 0. Recorded 7 frames.
Size: 0
Stopped recording episode 0. Recorded 9 frames.
Size: 7
learning
Stopped recording episode 0. Recorded 10 frames.
Size: 16
Stopped recording episode 0. Recorded 10 frames.
Size: 26
0
actions shape torch.Size([8, 5])
target value shape torch.Size([8, 6])
predicted values shape torch.Size([8, 6, 1])
target rewards shape torch.Size([8, 6])
predicted rewards shape torch.Size([8, 6, 1])
target to plays shape torch.Size([8, 6, 2])
predicted to_pl

/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


learning
100
actions shape torch.Size([8, 5])
target value shape torch.Size([8, 6])
predicted values shape torch.Size([8, 6, 1])
target rewards shape torch.Size([8, 6])
predicted rewards shape torch.Size([8, 6, 1])
target to plays shape torch.Size([8, 6, 2])
predicted to_plays shape torch.Size([8, 6, 2])
masks shape torch.Size([8, 6]) torch.Size([8, 6])
actions tensor([[1, 0, 0, 6, 5],
        [4, 0, 3, 2, 5],
        [2, 1, 0, 6, 5],
        [4, 0, 8, 6, 5],
        [2, 6, 3, 5, 7],
        [4, 3, 8, 6, 5],
        [6, 1, 0, 6, 5],
        [6, 1, 5, 0, 7]])
target value tensor([[-0.9900,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9900,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9227,  0.9321, -0.9415, -0.9510,  0.9606,  0.9703],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.9900,  1.0000,  0.0000,  0

...

learning
5400
actions shape torch.Size([8, 5])
target value shape torch.Size([8, 6])
predicted values shape torch.Size([8, 6, 1])
target rewards shape torch.Size([8, 6])
predicted rewards shape torch.Size([8, 6, 1])
target to plays shape torch.Size([8, 6, 2])
predicted to_plays shape torch.Size([8, 6, 2])
masks shape torch.Size([8, 6]) torch.Size([8, 6])
actions tensor([[7, 0, 4, 0, 0],
        [5, 8, 4, 0, 0],
        [4, 2, 7, 0, 3],
        [6, 1, 3, 0, 0],
        [3, 5, 4, 6, 0],
        [1, 0, 2, 0, 0],
        [6, 4, 7, 2, 8],
        [4, 0, 1, 7, 0]])
target value tensor([[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.9801,  0.9900,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9606, -0.9703, -0.9801,  0.9900,  1.0000,  0.0000],
        [-0.9801, -0.9900,  1.0000,  0.0000,  0.0000,  0.0000],
        [-0.9703, -0.9801,  0.9900,  1.0000,  0.0000,  0.0000],
        [-0.9801, -0.9900,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9510,  0.9606, -0.9703, -

/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


learning
5500
actions shape torch.Size([8, 5])
target value shape torch.Size([8, 6])
predicted values shape torch.Size([8, 6, 1])
target rewards shape torch.Size([8, 6])
predicted rewards shape torch.Size([8, 6, 1])
target to plays shape torch.Size([8, 6, 2])
predicted to_plays shape torch.Size([8, 6, 2])
masks shape torch.Size([8, 6]) torch.Size([8, 6])
actions tensor([[7, 6, 3, 8, 0],
        [4, 8, 3, 1, 7],
        [8, 1, 5, 0, 3],
        [7, 6, 3, 0, 4],
        [5, 0, 4, 0, 3],
        [4, 7, 8, 3, 6],
        [5, 4, 8, 3, 6],
        [4, 2, 8, 1, 7]])
target value tensor([[ 0.9606, -0.9703, -0.9801,  0.9900,  1.0000,  0.0000],
        [ 0.9510,  0.9606, -0.9703, -0.9801,  0.9900,  1.0000],
        [-0.9801, -0.9900,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9321, -0.9415, -0.9510,  0.9606,  0.9703, -0.9801],
        [-0.9801,  0.9900,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9606, -0.9703, -0.9801,  0.9900,  1.0000,  0.0000],
        [-0.9510,  0.9606,  0.9703, -

/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


learning
5600
actions shape torch.Size([8, 5])
target value shape torch.Size([8, 6])
predicted values shape torch.Size([8, 6, 1])
target rewards shape torch.Size([8, 6])
predicted rewards shape torch.Size([8, 6, 1])
target to plays shape torch.Size([8, 6, 2])
predicted to_plays shape torch.Size([8, 6, 2])
masks shape torch.Size([8, 6]) torch.Size([8, 6])
actions tensor([[3, 2, 6, 4, 0],
        [2, 3, 7, 5, 1],
        [6, 8, 3, 0, 1],
        [4, 0, 3, 5, 1],
        [5, 4, 0, 0, 1],
        [4, 0, 7, 6, 2],
        [8, 6, 0, 7, 2],
        [3, 0, 2, 0, 1]])
target value tensor([[ 0.9703, -0.9801, -0.9900,  1.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9510,  0.9606, -0.9703, -0.9801,  0.9900,  1.0000],
        [-0.9900,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9606, -0.9703, -0.9801,  0.9900,  1.0000,  0.0000],
        [ 0.9510,  0.9606, -0.9703, -